In [14]:
from optimum.onnxruntime import ORTModelForFeatureExtraction
from sentence_transformers import SentenceTransformer
from models import pipeline_onnx_transformers as pipe
from transformers import AutoTokenizer
from datetime import datetime
import numpy as np

In [2]:
model_st = SentenceTransformer.load('../models/zero-shot')

In [3]:
test_list = ['This is a test string for testing embedding speed.'] * 1000

In [4]:
duration_list = list()
for i in range(100):
    start = datetime.now()
    model_st.encode(test_list, convert_to_tensor=True)
    duration = datetime.now() - start
    duration_list.append(duration)

print('Average time for 1.000 strings: ', str(np.mean(duration_list)))
print('Median time for 1.000 strings: ', str(np.median(duration_list)))

Average time for 1.000 strings:  0:00:01.397364
Median time for 1.000 strings:  0:00:01.349685


In [9]:
onnx_model = ORTModelForFeatureExtraction.from_pretrained('../models/zero-shot', from_transformers=True)
onnx_tokenizer = AutoTokenizer.from_pretrained('../models/zero-shot')

In [11]:
vanilla_emb = pipe.SentenceEmbeddingPipeline(model=onnx_model, tokenizer=onnx_tokenizer)

In [21]:
duration_list = list()
for i in range(10):
    start = datetime.now()
    vanilla_emb(test_list)
    duration = datetime.now() - start
    duration_list.append(duration)

print('Average time for 1.000 strings: ', str(np.mean(duration_list)))
print('Median time for 1.000 strings: ', str(np.median(duration_list)))

Average time for 1.000 strings:  0:00:02.091777
Median time for 1.000 strings:  0:00:02.031237


In [17]:
start = datetime.now()
vanilla_emb(test_list)
print(datetime.now() - start)

0:00:02.085814


In [18]:
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

# create ORTOptimizer and define optimization configuration
optimizer = ORTOptimizer.from_pretrained(onnx_model)
optimization_config = OptimizationConfig(optimization_level=99) # enable all optimizations

# apply the optimization configuration to the model
optimizer.optimize(
    save_dir='../models/onnx',
    optimization_config=optimization_config,
)

AttributeError: 'OptimizationConfig' object has no attribute 'use_multi_head_attention'

In [20]:
onnx_model = ORTModelForFeatureExtraction.from_pretrained('../models/onnx')

NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from ../models/onnx/model.onnx failed:Load model ../models/onnx/model.onnx failed. File doesn't exist